In [54]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import math


doc1 = open("C:/Users/antoc/Downloads/PortableGit/practica2024/lcparam_full_long.txt")
lineas = doc1.readlines()

# Divide cada línea en columnas usando espacio como separador
datos_separados = [linea.strip().split(' ') for linea in lineas]

datos= []
for fila in datos_separados:
    datos.append(fila)
    #print(fila)
doc1.close()

z_cmb = []
for i in range(1,len(datos)):
  a= float(datos[i][1])
  z_cmb.append(a)

array_z_cmb = np.array(z_cmb)

m_b = []
for i in range(1, len(datos)):
  a= float(datos[i][4])
  m_b.append(a)


dm_b = []
for i in range(1, len(datos)):
    a= float(datos[i][5])
    dm_b.append(a)

#Relación magnitud aparente y absoluta con distancia de luminosidad
M = -19.3
D_L = np.zeros(len(m_b))
for i in range(len(m_b)):
  D_L[i]= 10**((m_b[i]-M-25)/5)
  
#print('D_L:', D_L)


In [55]:
#utilizando propagación de errores
dD = []
for i in range(len(dm_b)):
  a= (2**((m_b[i]-M-25)/5)) * (5**(((m_b[i]-M-25)/5)-1))* np.log(10) * dm_b[i]
  dD.append(a)

Datos de Hubble

In [56]:
doc2 = open("C:/Users/antoc/Downloads/PortableGit/practica2024/hub_data.txt")
lineas2 = doc2.readlines()

# Divide cada línea en columnas usando espacio como separador
datos_separados2 = [linea.strip().split(' ') for linea in lineas2]

datos= []
for fila in datos_separados2:
    datos.append(fila)

z = []
for i in range(len(datos)):
      a = float(datos[i][0])
      z.append(a)
array_z =np.array(z)

H = []
for i in range(len(datos)):
   if datos[i][1] == '':
      a = float(datos[i][2])
      H.append(a)
   else:
      a = float(datos[i][1])
      H.append(a)
array_H = np.array(H)

desv = []
for i in range(len(datos)):
    a = float(datos[i][2])
    desv.append(a)


In [57]:
H_0= 70
c = 3e6

E = np.zeros_like(H)
for i in range(len(H)):
    E[i] = H[i]/H_0

def regla_trapecio_numerica(x, y, j):
    integral = 0
    n = j
    for i in range(1, n):
        integral += (x[i] - x[i - 1]) * (y[i] + y[i - 1])/2

    return integral

# Calcular la aproximación de la integral utilizando la regla del trapecio
resultado_integral = np.zeros_like(H)
for j in range(len(H)):
    resultado_integral[j] = regla_trapecio_numerica(z, 1/E, j)
    #print("El resultado de la integral es:", resultado_integral[j])

DL_H = np.zeros_like(H)
for i in range(len(H)):
    DL_H[i]= c * (1+ z[i])/ H_0 * resultado_integral[i]


¿Como sacamos el error del DL a partir de la desviación estándar de H? (Lo hice con H(z) mientras)

In [84]:
def exp(x):
    return np.exp(x)


def ln(x):
    return np.log(x)
def sin(x):
    return np.sin(x)
def cos(x):
    return np.cos(x)
def senh(x):
    return np.sinh(x)
def cosh(x):
    return np.cosh(x)
def poli1(x):
    return x 
def poli2(x):
    return x**2 + x
def poli3(x):
    return x**3 + x**2 + x

grammar = [exp, ln, sin, cos, senh, cosh, poli1, poli2, poli3]
operaciones = ['+', '-', '*', '/']

#A partir de las funciones ingresadas inicialmente, calcula el chi_2
def chi_2_valores(func):
    return ((array_H - func(array_z))/desv)**2

def chi_2_f(func):
    chi_2_val = np.zeros_like(func)
    chi_2 = np.zeros_like(func)
    for i in range(len(func)):
        chi_2_val[i] = chi_2_valores(func[i])
        for j in range(len(chi_2_val)):
            chi_2[j] = np.sum(chi_2_val[j])
    
    return chi_2

print(chi_2_f(grammar))
chi_inic = chi_2_f(grammar)

[4157.01591713639 4430.910574765109 4336.577913283831 4323.031657922317
 4288.763494821255 4243.17294910014 4318.165617110693 4243.243296737529
 4110.323934510578]


In [89]:
tasa_seleccion= 1
cant_sobrevivientes = tasa_seleccion*len(chi_inic)
cant_sobreviv = int(cant_sobrevivientes)
print(cant_sobreviv)


chi_menores = sorted(chi_inic)[:cant_sobreviv]


9


In [ ]:
def funcion_combinada(func1, func2):
        for i in range(len(func1)):
            if operaciones[i] == '+':
                for j in range(len(cant_sobreviv)):
                    combinada = func1 + func2

            elif operaciones[i] == '-':
                resultado -= funciones[i](x)
            elif operaciones[i] == '*':
                resultado *= funciones[i](x)
            elif operaciones[i] == '/':
                resultado /= funciones[i](x)
        return resultado
    
    return funcion_combinada



In [59]:
'''
para asignar un peso a cada función
'''

def combinar_funciones(grammar, pesos):
    def funcion_combinada(x):
        resultado = 0.0
        for i, funcion in enumerate(grammar):
            resultado += pesos[i] * funcion(x)
        return resultado
    
    return funcion_combinada

# Asigna pesos inversamente proporcionales a los chi cuadrado
pesos = 1.0 / chi_2

# Crea la función combinada
funcion_combinada = combinar_funciones(grammar, pesos)

# Evalúa la función combinada en un punto
x_evaluado = 1.0
resultado_combinado = funcion_combinada(x_evaluado)

print("Resultado combinado en", x_evaluado, "es:", resultado_combinado)

Resultado combinado en 1.0 es: 0.2682941969615793


In [ ]:
def combinar_funciones(funcion_externa, funcion_interna):
    """
    Combina dos funciones, donde la función_interna se evalúa dentro de la función_externa.
    
    Parámetros:
    - funcion_externa: La función principal.
    - funcion_interna: La función que se evaluará dentro de la función principal.
    
    Retorna:
    - La función combinada.
    """
    def funcion_combinada(x):
        resultado_interno = funcion_interna(x)
        resultado_externo = funcion_externa(resultado_interno)
        return resultado_externo
    
    return funcion_combinada